# Обучаем Qwen

## Установка зависимостей

In [1]:
!git clone --depth 1 https://github.com/hiyouga/LLaMA-Factory.git

fatal: destination path 'LLaMA-Factory' already exists and is not an empty directory.


In [2]:
!pip uninstall -y jax
!pip install -e .[bitsandbytes,liger-kernel]
!pip install pandas
!pip install huggingface_hub
# Если в docker нету torch
# !pip install torch==2.3.1 torchvision==0.18.1 torchaudio==2.3.1
# !pip install -e ".[torch,metrics]"

Defaulting to user installation because normal site-packages is not writeable
Obtaining file:///home/jupyter/workspace
ERROR: file:///home/jupyter/workspace does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [3]:
%cd LLaMA-Factory

/home/jupyter/workspace/LLaMA-Factory


In [4]:
!pip install -r requirements.txt
!pip install llamafactory

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [5]:
%cd ..

/home/jupyter/workspace


Проверяем что нам доступен torch

In [6]:
import torch
try:
  assert torch.cuda.is_available() is True
except AssertionError:
  print("Please set up a GPU before using LLaMA Factory: https://medium.com/mlearning-ai/training-yolov4-on-google-colab-316f8fff99c6")

## Поменять при необходимости

Указываем модель и для какой задачи будет использоваться

**task_description должен совпадать с названием датасета**

In [11]:
model="Qwen/Qwen2.5-7B-Instruct"
task_description = "matching"

## Устанавливаем модель

Обязательно делаем это тут, а не в LLaMA Factory, так как там это синхроно

In [8]:
from huggingface_hub import login, snapshot_download

login("hf_token")

snapshot_download(model)

Fetching 14 files:   0%|          | 0/14 [00:00<?, ?it/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

LICENSE:   0%|          | 0.00/11.3k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/663 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/27.8k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

'/home/jupyter/.cache/huggingface/hub/models--Qwen--Qwen2.5-7B-Instruct/snapshots/a09a35458c702b33eeacc393d103063234e8bc28'

Папка для сохранения результата

In [9]:
from datetime import datetime
import os

timestamp = datetime.now().strftime("%Y-%m-%d-%H:%M:%S")
output_dir = f'./lora/{model}/{task_description}/train_{timestamp}'
# os.makedirs(output_dir, exist_ok=True)

## Запускаем обучение

Меняем на нужные параметры запускаем

In [10]:
step = 10
batch_size = 1
max_prompt_size = 10000
lora_rank = 8
lora_alpha = 16
lora_dropout = 0
save_steps = 5000


command = f"""
llamafactory-cli train \
    --stage sft \
    --do_train True \
    --model_name_or_path {model} \
    --preprocessing_num_workers 16 \
    --finetuning_type lora \
    --template qwen \
    --flash_attn auto \
    --dataset_dir ./ \
    --dataset {task_description} \
    --cutoff_len {max_prompt_size} \
    --learning_rate 5e-05 \
    --num_train_epochs 3.0 \
    --max_samples {step} \
    --per_device_train_batch_size {batch_size} \
    --gradient_accumulation_steps 8 \
    --lr_scheduler_type cosine \
    --max_grad_norm 1.0 \
    --logging_steps 5 \
    --save_steps {save_steps} \
    --warmup_steps 0 \
    --packing False \
    --report_to none \
    --output_dir {output_dir} \
    --bf16 True \
    --plot_loss True \
    --ddp_timeout 180000000 \
    --optim adamw_torch \
    --lora_rank {lora_rank} \
    --lora_alpha {lora_alpha} \
    --lora_dropout {lora_dropout} \
    --lora_target all
"""

# Выполняем команду
get_ipython().system(command)


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.
0it [00:00, ?it/s]
[INFO|2025-01-25 10:10:41] llamafactory.hparams.parser:355 >> Process rank: 0, device: cuda:0, n_gpu: 1, distributed training: False, compute dtype: torch.bfloat16
[INFO|configuration_utils.py:679] 2025-01-25 10:10:41,731 >> loading configuration file config.json from cache at /home/jupyter/.cache/huggingface/hub/models--Qwen--Qwen2.5-7B-Instruct/snapshots/a09a35458c702b33eeacc393d103063234e8bc28/config.json
[INFO|configuration_utils.py:746] 2025-01-25 10:10:41,738 >> Model config Qwen2Config {
  "_name_or_path": "Qwen/Qwen2.5-7B-Instruct",
  "architectures": [
    "Qwen2ForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 151643,
  "eos_token_id": 151645,
  "hidden_act": "silu",
  "hidden_size": 3584,
  "initializer_range": 0.0

In [ ]:
!GRADIO_SHARE=1 llamafactory-cli webui